In [5]:
#import libraries
import pandas as pd
import numpy as np
import bs4 as soup
import html5lib, lxml

#web scrape html
post_codes = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

#clean data
df_codes = pd.DataFrame(post_codes[0]) #save correct table to df
df_codes = df_codes.replace('Not assigned', np.nan) #replace 'Not assigned' with true NaN value
df_codes.dropna(subset=['Borough'], axis=0, inplace=True) #drop all rows with NaN value in the Borough column

geo_data = pd.read_csv('https://cocl.us/Geospatial_data')

merged_df = df_codes.merge(geo_data, how='outer')